In [13]:
!pip install gradio

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = 

In [4]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer
from openai import OpenAI
from huggingface_hub import login
import os
from IPython.display import Markdown, display

In [5]:
# Constants
DEEPSEEK = "deepseek-ai/deepseek-coder-6.7b-base"
AUDIO_MODEL = "whisper-1"

In [8]:
def process_audio(audio_path, openai_api_key):
    """
    Process audio file and generate meeting minutes
    """
    try:
        # Initialize OpenAI client
        openai_client = OpenAI(api_key=openai_api_key)

        # Transcribe audio
        with open(audio_path, "rb") as audio_file:
            transcription = openai_client.audio.transcriptions.create(
                model=AUDIO_MODEL,
                file=audio_file,
                response_format="text"
            )

        # Use OpenAI for generating minutes as well
        system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
        user_prompt = f"Below is an extract transcript of a meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt}
            ]
        )

        minutes = response.choices[0].message.content

        return minutes, transcription

    except Exception as e:
        return f"An error occurred: {str(e)}", "Transcription failed"

In [12]:
def create_gradio_interface():
    """
    Create and launch the Gradio interface
    """
    # Define the interface
    with gr.Blocks() as ui:
        gr.Markdown("# Meeting Minutes Generator")

        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(
                    label="Upload Audio File",
                    type="filepath"
                )
                openai_key = gr.Textbox(
                    label="OpenAI API Key",
                    type="password"
                )
                process_btn = gr.Button("Generate Minutes")

            with gr.Column():
                minutes_output = gr.Markdown(
                    label="Meeting Minutes"
                )
                transcription_output = gr.Textbox(
                    label="Transcription",
                    lines=10

        # Connect the interface
        process_btn.click(
            fn=process_audio,
            inputs=[audio_input, openai_key],
            outputs=[minutes_output, transcription_output]
        )

        gr.Markdown("""
        ## Instructions
        1. Upload an audio file of your meeting
        2. Enter your OpenAI API key
        3. Click 'Generate Minutes' to process the audio

        Note: This tool has no access to your OPENAI API KEY. This tool uses OpenAI's Whisper for transcription and GPT-3.5 for minutes generation.
        """)

    # Launch the interface
    ui.launch(share=True)

if __name__ == "__main__":
    create_gradio_interface()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86bd3892db2d7551da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86bd3892db2d7551da.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
